In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pytorch-tabnet==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import torch

from torch import nn
from pytorch_tabnet.tab_model  import TabNetRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
covid = pd.read_csv('/content/drive/MyDrive/covid/covid.csv',encoding = 'cp949')

In [ ]:
kor_df = covid[covid['iso_code']=='KOR']
kor_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
156083,KOR,Asia,South Korea,2020-01-22,1.0,NaN,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156084,KOR,Asia,South Korea,2020-01-23,1.0,0.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156085,KOR,Asia,South Korea,2020-01-24,2.0,1.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156086,KOR,Asia,South Korea,2020-01-25,2.0,0.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156087,KOR,Asia,South Korea,2020-01-26,3.0,1.0,NaN,NaN,NaN,NaN,...,6.2,40.9,NaN,12.27,83.03,0.916,43.4,0.17,2.47,0.845918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
156923,KOR,Asia,South Korea,2022-05-11,17694677.0,35883.0,36658.429,23554.0,63.0,56.571,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156924,KOR,Asia,South Korea,2022-05-12,17727086.0,32409.0,37472.000,23606.0,52.0,57.143,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156925,KOR,Asia,South Korea,2022-05-13,17756627.0,29541.0,36041.857,23661.0,55.0,53.143,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN
156926,KOR,Asia,South Korea,2022-05-14,17782061.0,25434.0,33951.857,23709.0,48.0,49.857,...,6.2,40.9,NaN,12.27,83.03,0.916,NaN,NaN,NaN,NaN


In [ ]:
cols = ['total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'icu_patients', 'total_tests','new_tests','total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 
        'total_boosters', 'new_vaccinations']
kor = kor_df[cols]
kor

,total_cases,new_cases,total_deaths,new_deaths,icu_patients,total_tests,new_tests,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations
156083,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156084,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156085,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156086,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156087,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
156923,17694677.0,35883.0,23554.0,63.0,NaN,46684375.0,158366.0,124478764.0,45041195.0,44559167.0,36394648.0,107161.0
156924,17727086.0,32409.0,23606.0,52.0,NaN,NaN,NaN,124583298.0,45041901.0,44560095.0,36497550.0,104534.0
156925,17756627.0,29541.0,23661.0,55.0,NaN,NaN,NaN,124746652.0,45043384.0,44562056.0,36657464.0,163354.0
156926,17782061.0,25434.0,23709.0,48.0,NaN,NaN,NaN,124783300.0,45044088.0,44562791.0,36692674.0,36648.0


In [ ]:
kor = kor.fillna(0)
kor

,total_cases,new_cases,total_deaths,new_deaths,icu_patients,total_tests,new_tests,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations
156083,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156084,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156085,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156086,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
156087,3.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
156923,17694677.0,35883.0,23554.0,63.0,0.0,46684375.0,158366.0,124478764.0,45041195.0,44559167.0,36394648.0,107161.0
156924,17727086.0,32409.0,23606.0,52.0,0.0,0.0,0.0,124583298.0,45041901.0,44560095.0,36497550.0,104534.0
156925,17756627.0,29541.0,23661.0,55.0,0.0,0.0,0.0,124746652.0,45043384.0,44562056.0,36657464.0,163354.0
156926,17782061.0,25434.0,23709.0,48.0,0.0,0.0,0.0,124783300.0,45044088.0,44562791.0,36692674.0,36648.0


In [ ]:
X = kor.drop('new_cases', axis=1)
y = kor['new_cases']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_X = X_train.to_numpy()
train_y = y_train.to_numpy().reshape(-1,1)
test_X = X_test.to_numpy()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaler = scaler.fit_transform(train_X)
X_test_scaler = scaler.fit_transform(test_X)

In [ ]:
kf = KFold(n_splits=5, random_state=42, shuffle=True)
scores  = []

for trn_idx,val_idx in kf.split(train_X):
    X_trn = X_train_scaler[trn_idx]
    y_trn = train_y[trn_idx]
    X_val = X_train_scaler[val_idx]
    y_val = train_y[val_idx]
    
    model = TabNetRegressor(verbose = 1,seed = 42,optimizer_fn=torch.optim.AdamW)
    model.fit(X_train=X_trn, y_train=y_trn,
              eval_set=[(X_val, y_val)],
            patience=100, max_epochs=2000,
              eval_metric=['rmse'])
    scores.append(model.best_cost)
    
    pred = model.predict(X_test_scaler)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
epoch 1044| loss: 3922908160.0| val_0_rmse: 36732.00726|  0:00:43s
epoch 1045| loss: 3912602368.0| val_0_rmse: 36592.89194|  0:00:43s
epoch 1046| loss: 3904409856.0| val_0_rmse: 36167.63688|  0:00:43s
epoch 1047| loss: 3911006208.0| val_0_rmse: 36185.28013|  0:00:43s
epoch 1048| loss: 3979663616.0| val_0_rmse: 36526.51153|  0:00:43s
epoch 1049| loss: 3951394304.0| val_0_rmse: 36940.09269|  0:00:43s
epoch 1050| loss: 3972961792.0| val_0_rmse: 36785.48448|  0:00:43s
epoch 1051| loss: 3916377856.0| val_0_rmse: 36296.759|  0:00:43s
epoch 1052| loss: 3924934912.0| val_0_rmse: 35905.86795|  0:00:43s
epoch 1053| loss: 3940600832.0| val_0_rmse: 36011.37078|  0:00:43s
epoch 1054| loss: 3911391488.0| val_0_rmse: 36324.92612|  0:00:43s
epoch 1055| loss: 3931126016.0| val_0_rmse: 36107.03466|  0:00:43s
epoch 1056| loss: 3940524800.0| val_0_rmse: 35550.37895|  0:00:43s
epoch 1057| loss: 3965913088.0| val_0_rmse: 34994.17556|  0:00:43s
epoch 1058| loss: 3916012032

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
MAE =  mean_absolute_error(y_test, pred)

MSE = mean_squared_error(y_test, pred)

RMSE = np.sqrt(MSE)

R2 = r2_score(y_test, pred)

cv = np.std(scores)/np.mean(scores)

print("MAE 점수 : %.5f" % MAE)
print("MSE 점수 : %.5f" % MSE)
print("RMSE 점수 : %.5f" % RMSE)
print("R2 점수 : %.5f" % R2)
print('CV계수 : %.5f' % cv)

MAE 점수 : 15306.89052
MSE 점수 : 2265118500.30200
RMSE 점수 : 47593.26108
R2 점수 : 0.62967
CV계수 : 0.20699


* 전체 컬럼을 학습시켰을 때 R2 점수는 약 31%
* EDA에서 분류한 컬럼들로만 학습 시켰을 때 R2점수는 약 63%
* NaN값을 제거 후 데이터가 많이 줄어들어 학습 시켰을 때 오히려 R2점수는 오히려 하락함